# Практическое занятие №4

Выполнил: *Грибков А.С.*

Группа: *ИКБО-16-22*

## 1. Некоторые операции с классами и объектами

**1.1.** (уровень сложности: простейший)

Напишите код, который выведет на экране все имена полей объекта произвольного пользовательского класса, кроме служебных имен.

**Решение:**

In [59]:
class Test:
    def __init__(self, field1, field2):
        self.field1 = field1
        self.field2 = field2

    def fun(self):
        print(self.field1, self.field2)


test_object = Test("1", "2")

In [60]:
list(filter(lambda s: len(s) < 2 or s[:2] != "__", dir(test_object)))

**1.2.** (уровень сложности: простейший)

Напишите код, который по имени метода, заданному строкой, вызовет этот метод в объекте некоторого пользовательского класса.

**Решение:**

In [61]:
getattr(test_object, "fun")()

**1.3.** (уровень сложности: простейший)

С кодом ниже что-то не так. Что именно неправильно и как это исправить?

```Python
class A:
    pass

class B(A):
    pass

class C(A, B):
    pass
```

**Решение:**

Ломается, т.к. несколько раз ```C``` наследуется от ```A``` (напрямую и через ```B```):

In [62]:
class A:
    pass


class B(A):
    pass


class C(A, B):
    pass

Для исправления необходимо убрать наследование ```C``` от ```A```. При этом, потерь не будет, т.к. все методы и поля ```A``` и так передаются в ```C``` через ```B```.

In [63]:
class A:
    pass


class B(A):
    pass


class C(B):
    pass

**1.4.** (уровень сложности: низкий)

Напишите функцию-однострочник get_inheritance для вывода строки, отражающей иерархию наследования для входного класса.

Пример:

```Python
>>> print(get_inheritance(OSError))
OSError -> Exception -> BaseException -> object
```

**Решение:**

In [64]:
" -> ".join(map(lambda x: x.__name__, OSError.__mro__))

## 2. Своя реализация структуры данных

Реализуйте хэш-таблицу, аналог встроенного dict. Используйте для внутренней реализации список пар ключ-значение. Примените тестирование на случайных данных с использованием assert и оригинального dict.

**2.1.** (уровень сложности: средний)

Реализуйте методы чтения, записи и получения размера хэш-таблицы.

**2.2.** (уровень сложности: низкий)

Реализуйте для методов своей хэш-таблицы тот же интерфейс, что и в dict, включая перегрузку операций.

**2.3.** (уровень сложности: средний)

Реализуйте поддержку итератора для цикла for. Обязательно протестируйте код на примерах с вложенными циклами!

**Решение:**

In [65]:
class HashTable:
    class Node:
        def __init__(self, key, value):
            self.key = key
            self.value = value
            self.deleted = False

    def __init__(self, init_capacity=50):
        self.size = init_capacity
        self.busy = 0
        self.items = [None] * init_capacity
        self.i = None
        self.i_stack = []

    def hash(self, key):
        return hash(key) % self.size

    def __setitem__(self, key, value):
        node = self.Node(key, value)
        node_hash = self.hash(key)

        while self.items[node_hash] is not None:
            node_hash = (node_hash + 1) % self.size

        self.items[node_hash] = node
        self.busy += 1

        if self.size * 0.75 < self.busy:
            self.update()

    def __getitem__(self, key):
        node_hash = self.hash(key)

        while self.items[node_hash] is not None:

            if self.items[node_hash].key == key:
                return self.items[node_hash]

            node_hash = (node_hash + 1) % self.size

        return None

    def update(self):
        self.size = self.busy * 2
        self.busy = 0
        save_items = self.items
        self.items = [None] * self.size

        for node in save_items:
            if node is not None:
                self[node.key] = node.value

    def __len__(self):
        return self.size

    def __iter__(self):
        if i is not None:
            self.i_stack.append(self.i)

        self.i = 0
        return self

    def __next__(self):
        self.i += 1
        while self.i < self.size:
            if self.items[self.i] is not None:
                return self.items[self.i].key, self.items[self.i].value
            self.i += 1

        if len(self.i_stack) != 0:
            self.i = self.i_stack.pop()
        raise StopIteration()


In [66]:
from random import randint

table = HashTable()
for i in range(2, 10):
    table[randint(1, i)] = randint(1, i)

for key, val in table:
    for key1, val1 in table:
        print(key, val, key1, val1)

## 3. Деревья выражений

Требуется реализовать операции для печати, вычисления арифметических выражений и компиляции выражений в код стековой машины.

**3.1.** (уровень сложности: низкий)

Реализовать классы узлов дерева: Num, Add и Mul. Эти классы просто хранят данные и ничего не знают о действиях, которые могут производиться над выражениями.

Пример, который будет использоваться далее:

```Python
ast = Add(Num(7), Mul(Num(3), Num(2)))
```

**Решение:**

In [67]:
class Operation:
    def calc(self):
        pass

    def get(self) -> str:
        pass

    def stack(self) -> str:
        pass


class Num(Operation):
    def __init__(self, value):
        self.value = value

    def calc(self):
        return self.value

    def get(self):
        return str(self.value)

    def stack(self):
        return "PUSH " + str(self.value)


class Add(Operation):
    def __init__(self, num1: Operation, num2: Operation):
        self.num1 = num1
        self.num2 = num2

    def calc(self):
        return self.num1.calc() + self.num2.calc()

    def get(self) -> str:
        return f"({self.num1.get()} + {self.num2.get()})"

    def stack(self) -> str:
        return f"{self.num1.stack()}\n{self.num2.stack()}\nADD"


class Mul(Operation):
    def __init__(self, num1: Operation, num2: Operation):
        self.num1 = num1
        self.num2 = num2

    def calc(self):
        return self.num1.calc() * self.num2.calc()

    def get(self) -> str:
        return f"({self.num1.get()} * {self.num2.get()})"

    def stack(self) -> str:
        return f"{self.num1.stack()}\n{self.num2.stack()}\nMUL"

In [68]:
ast = Add(Num(7), Mul(Num(3), Num(2)))

**3.2.** (уровень сложности: средний)

Реализовать класс-посетитель PrintVisitor для печати выражения. Обойтись без перегрузки repr и str, а также без операторов ветвления!

Пример:

```Python
>>> pv = PrintVisitor()
>>> print(pv.visit(ast))
(7 + (3 * 2))
```

**Решение:**

In [69]:
class PrintVisitor:
    def visit(self, expression: Operation):
        return expression.get()

In [70]:
pv = PrintVisitor()
print(pv.visit(ast))

**3.3.** (уровень сложности: средний)

Реализовать класс-посетитель CalcVisitor для вычисления выражения. Обойтись без eval с exec, а также без операторов ветвления!

Пример:

```Python
>>> cv = CalcVisitor()
>>> print(cv.visit(ast))
13
```

**Решение:**

In [71]:
class CalcVisitor:
    def visit(self, expression: Operation):
        return expression.calc()

In [72]:
cv = CalcVisitor()
print(cv.visit(ast))

**3.4.** (уровень сложности: средний)

Реализовать класс-посетитель StackVisitor для порождения кода стековой машины по выражению. Обойтись без операторов ветвления.

Пример:

```Python
>>> sv = StackVisitor()
>>> print(sv.visit(ast))
PUSH 7
PUSH 3
PUSH 2
MUL
ADD
```

**Решение:**

In [73]:
class StackVisitor:
    def visit(self, expression: Operation):
        return expression.stack()

In [74]:
sv = StackVisitor()
print(sv.visit(ast))

## 4. Предметно-ориентированный язык HTML-тегов

**4.1.** (уровень сложности: средний)

Реализовать язык HTML-тегов с помощью менеджера контекста.

Реализовать классы для выполнения следующего примера:

```Python
html = HTML()
with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')
```

С помощью html.get_code() выдается следующий результат:

```HTML
<body>
<div>
<div>
<p>Первая строка.</p>
<p>Вторая строка.</p>
</div>
<div>
<p>Третья строка.</p>
</div>
</div>
</body>
```

**Решение:**

In [75]:
from contextlib import contextmanager


class HTML:
    def __init__(self):
        self.code = ''

    @contextmanager
    def body(self):
        self.code += '<body>\n'
        yield
        self.code += '</body>\n'

    @contextmanager
    def div(self):
        self.code += '<div>\n'
        yield
        self.code += '</div>\n'

    def p(self, text):
        self.code += f'<p>{text}</p>\n'

    def get_code(self):
        return self.code

In [76]:
html = HTML()

with html.body():
    with html.div():
        with html.div():
            html.p('Первая строка.')
            html.p('Вторая строка.')
        with html.div():
            html.p('Третья строка.')

print(html.get_code())

## 5. Визуализация графов

**5.1.** (уровень сложности: средний)

Реализовать прототип библиотеки для рисования в формате SVG.

Для некоторых из следующих задач потребуется библиотека для работы с SVG-графикой. Достаточно поддержать только рисование линий и кругов с указанием цвета.

Пример:

```Python
svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='red')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='red')

svg.save('pic.svg', 100, 100)
```

Файл pic.svg будет иметь следующий вид:

```XML
<svg version="1.1" width="100.000000" height="100.000000" xmlns="http://www.w3.org/2000/svg">
<line x1="10.000000" y1="10.000000" x2="60.000000" y2="10.000000" stroke="black" />
<line x1="60.000000" y1="10.000000" x2="60.000000" y2="60.000000" stroke="black" />
<line x1="60.000000" y1="60.000000" x2="10.000000" y2="60.000000" stroke="black" />
<line x1="10.000000" y1="60.000000" x2="10.000000" y2="10.000000" stroke="black" />
<circle cx="10.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="10.000000" r="5.000000" fill="red" />
<circle cx="60.000000" cy="60.000000" r="5.000000" fill="red" />
<circle cx="10.000000" cy="60.000000" r="5.000000" fill="red" />
</svg>
```

**Решение:**

In [77]:
class SVG:
    def __init__(self):
        self.code = ''

    def line(self, x1, y1, x2, y2, color):
        self.code += f'<line x1="{x1}" y1="{y1}" x2="{x2}" y2="{y2}" stroke="{color}" />\n'
        
    def circle(self, cx, cy, r, color):
        self.code += f'<circle cx="{cx}" cy="{cy}" r="{r}" fill="{color}" />\n'
        
    def save(self, filename, width, height):
        self.code = f'<svg version="1.1" width="{width}" height="{height}" xmlns="http://www.w3.org/2000/svg">\n{self.code}</svg>'
        
        f = open(f"part5/{filename}", "w")
        f.write(self.code)
        f.close()

In [78]:
svg = SVG()

svg.line(10, 10, 60, 10, color='black')
svg.line(60, 10, 60, 60, color='black')
svg.line(60, 60, 10, 60, color='black')
svg.line(10, 60, 10, 10, color='black')

svg.circle(10, 10, r=5, color='red')
svg.circle(60, 10, r=5, color='red')
svg.circle(60, 60, r=5, color='red')
svg.circle(10, 60, r=5, color='red')

svg.save('pic.svg', 100, 100)

print(svg.code)

Картинка:

![](part5/pic.svg)